In [3]:
import time, re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [4]:
# Define numero aleatorios entre 3 y 7
def distriProb_lognorm():
  pdf = np.random.lognormal(mean=2.0, sigma=0.5, size=(1))
  if pdf<3  or pdf>7:
    pdf=np.random.randint(3,6)
  return float(pdf)

In [6]:
# Define el nevegador
options = webdriver.ChromeOptions()
#options.add_argument('--headless')
#options.add_argument('--no-sandbox')
#options.add_argument('--disable-dev-shm-usage')
chrome_driver = "./chromedriver"
options.binary_location = "/usr/bin/google-chrome-stable"
webpage_target = 'https://www.sciencedirect.com/'
driver = webdriver.Chrome(options = options)

In [72]:
# Colocamos el link de la pagina donde queremos hacer la busqueda
try:
    driver.get(webpage_target)
except:
    print('Hubo un error de conexion intentando de nuevo...')
    time.sleep(distriProb_lognorm())
    driver.refresh()
    driver.get(webpage_target)

In [73]:
# Enviamos un query de busqueda a la barra de busqueda
search_box_id = 'qs'
query = 'eddies'
search_box = driver.find_element(By.ID, search_box_id)
search_box.send_keys(query)
search_box.send_keys(Keys.RETURN) #click enter

In [74]:
# Filtramos los resultados por los que corresponden a reviewa
try:
    time.sleep(distriProb_lognorm())
    reviews = driver.find_element(By.ID, "articleTypes-REV")
    driver.execute_script("arguments[0].click();", reviews)
except:
    print('Hubo un error de conexion intentando de nuevo...')
    time.sleep(10)
    driver.refresh()
    reviews = driver.find_element(By.ID, "articleTypes-REV")
    driver.execute_script("arguments[0].click();", reviews)
try:
    time.sleep(distriProb_lognorm())
    earth = driver.find_element(By.ID, "subjectAreas-1900")
    driver.execute_script("arguments[0].click();", earth)
except:
    print('Hubo un error de conexion intentando de nuevo...')
    time.sleep(10)
    driver.refresh()
    time.sleep(10)
    show_more = driver.find_element(By.CSS_SELECTOR, '[aria-label="Show more subjectAreas filters"]')
    driver.execute_script("arguments[0].click();", show_more)
    time.sleep(distriProb_lognorm())
    earth = driver.find_element(By.ID, "subjectAreas-1900")
    driver.execute_script("arguments[0].click();", earth)

Hubo un error de conexion intentando de nuevo...


In [75]:
# Definimos 100 articulos por pagina
time.sleep(distriProb_lognorm())
xpath_100 = "//a[@class='anchor' and @data-aa-name='srp-100-results-per-page']"
first_100 = driver.find_element(By.XPATH, xpath_100)
driver.execute_script("arguments[0].click();", first_100)

/tmp/ipykernel_11566/140653954.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(pdf)


In [76]:
# Extreamos el html
time.sleep(distriProb_lognorm())
soup = BeautifulSoup(driver.page_source, 'html.parser')
art_html = soup.find_all('a', 'anchor result-list-title-link u-font-serif text-s anchor-default', 'rev')
pages_html = soup.find(id="srp-pagination")
n_pg = re.findall(r'Page\s\d of\s(\d+)', str(pages_html))[0]
link_list = [link.get('href') for link in art_html]
# Itera sobre cada pagina de los resultados de busqueda
for _ in range(int(n_pg)-1):
    # Siguiente pagina
    time.sleep(distriProb_lognorm())
    xpath_next = "//a[@class='anchor' and @data-aa-name='srp-next-page']"
    next_100 = driver.find_element(By.XPATH, xpath_next)
    driver.execute_script("arguments[0].click();", next_100)
    # Extreamos el html
    time.sleep(distriProb_lognorm())
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    art_html = soup.find_all('a', 'anchor result-list-title-link u-font-serif text-s anchor-default', 'rev')
    # Extreamos el link del html
    next_link_list = [link.get('href') for link in art_html]
    # Se añade a una lista
    link_list.extend(next_link_list)
    print(len(link_list))

200
300


/tmp/ipykernel_11566/140653954.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(pdf)


400
500
600
700
800
900
1000
1019


In [77]:
df = pd.DataFrame({'link': link_list})
df['link'] = df['link'].map(lambda link: 'https://www.sciencedirect.com'+link)
abstracts = []
titles = []
for link in df['link']:
    #driver = webdriver.Chrome(executable_path = chrome_driver, options = options)
    time.sleep(distriProb_lognorm())
    driver.get(link)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    html_title = soup.find_all('span', 'title-text')
    title_text = re.findall(r"(?<=\>).+(?=<)", str(html_title))
    titles.extend(title_text)
    rexp = re.compile(r"(?<=\d\"\>)\w[A-Za-z\s$&+,:;=?@#|'.^*()%!-’]+\.")
    html_abstract = soup.find_all('div', 'abstract author')
    abstract_text = re.findall(rexp, str(html_abstract))
    abstracts.extend(abstract_text)
    print(len(abstracts), end='\r')
df['titles'] = titles
df['abstracts'] = abstracts
df.to_csv('abstract_pln.csv')

/tmp/ipykernel_11566/140653954.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(pdf)
